In [4]:
import pandas as pd
import calendar
import os
import re

# Folder utama
base_folder = r"C:\Users\Helga\semenindonesia.com\admpbp - PRODUKSI"

# Sheet yang ingin dibaca
kiln_sheets = ["Kiln 1", "Kiln 2", "Kiln 3", "Kiln 4"]

# Header kolom
custom_header = [
    "Date", "Production (TPD)", "Production (TPH)", "Power (KWH/T)",
    "Heat (Kcal/Kg)", "Pemakaian BB (%)", "Running Time (Hrs)",
    "Number of Stops", "Difference", "Cause of stop", "Aktifitas ikutan",
    "ME", "EL", "INS", "PROC", "QUA", "DL/SL/QA", "Penyebab < 8000 T/day",
    "Keterangan Pakai IDO (1)", "Klt (1)", "Keterangan Pakai IDO (2)",
    "Klt (2)", "AFR", "Ket TSR", "JOP"
]

# Gabungan data
all_data = []

# Pola nama file yang valid
valid_pattern = re.compile(r"^Tuban kiln operation report \d{2}-\d{4}\.xlsx$", re.IGNORECASE)

# Hanya folder valid (JAN, PEB, dst) yang diproses
bulan_folder_valid = {"JAN", "PEB", "MAR", "APR", "MEI", "JUN", "JUL", "AGU", "SEP", "OKT", "NOV", "DES"}

# Loop hanya subfolder 1 tingkat
for subfolder in os.listdir(base_folder):
    subfolder_path = os.path.join(base_folder, subfolder)
    if not os.path.isdir(subfolder_path):
        continue

    # Cek apakah nama folder sesuai dengan daftar bulan
    if subfolder.upper() not in bulan_folder_valid:
        continue

    # Cek semua file di folder bulan
    for file in os.listdir(subfolder_path):
        if file.endswith(".xlsx") and valid_pattern.match(file):
            full_path = os.path.join(subfolder_path, file)

            # Ambil bulan dan tahun dari nama file
            match = re.search(r"(\d{2})-(\d{4})", file)
            if not match:
                print(f" -- Lewati (format tidak cocok): {file}")
                continue

            bulan_num = int(match.group(1))
            tahun = int(match.group(2))
            jumlah_hari = calendar.monthrange(tahun, bulan_num)[1]
            nama_bulan = calendar.month_name[bulan_num]

            for kiln in kiln_sheets:
                try:
                    df = pd.read_excel(
                        full_path,
                        sheet_name=kiln,
                        skiprows=7,
                        usecols="F:AD",
                        nrows=jumlah_hari,
                        header=None
                    )

                    df.columns = custom_header
                    df["Date"] = pd.to_numeric(df["Date"], errors="coerce")
                    df = df[df["Date"].between(1, jumlah_hari)]

                    df["tanggal"] = pd.to_datetime({
                        "year": tahun,
                        "month": bulan_num,
                        "day": df["Date"].astype(int)
                    })

                    df["KILN"] = kiln
                    df["Bulan"] = nama_bulan
                    df["SourceFilePath"] = full_path

                    all_data.append(df)
                    print(f" -- Dibaca: {file} - {kiln}")

                except Exception as e:
                    print(f" -- Gagal membaca {file} - {kiln}: {e}")

# Gabungkan & simpan
if all_data:
    hasil = pd.concat(all_data, ignore_index=True)
    hasil.sort_values(by=["tanggal", "KILN"], inplace=True)

    output_path = r"D:\MAGANG SIG\TUGAS 1\MERGE AWAL\auto_jan-jul.xlsx"
    hasil.to_excel(output_path, index=False)
    print(f"\n -- Berhasil disimpan di: {output_path}")
else:
    print("\n -- Tidak ada data yang berhasil dibaca.")


 -- Dibaca: Tuban kiln operation report 04-2025.xlsx - Kiln 1
 -- Dibaca: Tuban kiln operation report 04-2025.xlsx - Kiln 2
 -- Dibaca: Tuban kiln operation report 04-2025.xlsx - Kiln 3
 -- Dibaca: Tuban kiln operation report 04-2025.xlsx - Kiln 4
 -- Dibaca: Tuban kiln operation report 01-2025.xlsx - Kiln 1
 -- Dibaca: Tuban kiln operation report 01-2025.xlsx - Kiln 2
 -- Dibaca: Tuban kiln operation report 01-2025.xlsx - Kiln 3
 -- Dibaca: Tuban kiln operation report 01-2025.xlsx - Kiln 4
 -- Dibaca: Tuban kiln operation report 07-2025.xlsx - Kiln 1
 -- Dibaca: Tuban kiln operation report 07-2025.xlsx - Kiln 2
 -- Dibaca: Tuban kiln operation report 07-2025.xlsx - Kiln 3
 -- Dibaca: Tuban kiln operation report 07-2025.xlsx - Kiln 4
 -- Dibaca: Tuban kiln operation report 06-2025.xlsx - Kiln 1
 -- Dibaca: Tuban kiln operation report 06-2025.xlsx - Kiln 2
 -- Dibaca: Tuban kiln operation report 06-2025.xlsx - Kiln 3
 -- Dibaca: Tuban kiln operation report 06-2025.xlsx - Kiln 4
 -- Diba